<a href="https://colab.research.google.com/github/Boss-Darius/PMP-2023/blob/main/PMP_Lab_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt


np.random.seed(1)

# 1
clusters = 3
n_cluster = [200, 150, 150]
n_total = sum(n_cluster)
means = [5, 0, -5]
std_devs = [2, 2, 2]
mix = np.concatenate([np.random.normal(means[i], std_devs[i], size=n_cluster[i]) for i in range(clusters)])

# Plot the generated data
az.plot_kde(np.array(mix))
plt.title('Generated Data')
plt.show()
#2
for num_components in range(2, 5):
    with pm.Model() as model:
        means = pm.Normal('means', mu=[0] * num_components, sd=10, shape=num_components)

        std_devs = pm.HalfNormal('std_devs', sd=10, shape=num_components)

        weights = pm.Dirichlet('weights', a=np.ones(num_components))

        obs = pm.NormalMixture('obs', w=weights, mu=means, sd=std_devs, observed=mix)

        trace = pm.sample(1000, tune=1000)

    # Display the traceplot
    pm.traceplot(trace, var_names=['means', 'std_devs', 'weights'])
    plt.show()

    # 3
    waic = pm.waic(trace, model)
    loo = pm.loo(trace, model)

    print(f"Number of Components: {num_components}")
    print("WAIC:", waic.waic)
    print("LOO:", loo.loo)